In [1]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import torch.utils.data

In [3]:
ls

CNN_character.ipynb  DBP_wiki_data.csv  README.md   labels_index.pkl  texts.pkl
CNN_text.ipynb       GLOVE/             labels.pkl  newsgroups/


In [4]:
data=pd.read_csv('DBP_wiki_data.csv')

In [5]:
data.head()

,text,l1,l2,l3,wiki_name,word_count
0,The 1994 Mindoro earthquake occurred on Novemb...,Event,NaturalEvent,Earthquake,1994_Mindoro_earthquake,59
1,The 1917 Bali earthquake occurred at 06:50 loc...,Event,NaturalEvent,Earthquake,1917_Bali_earthquake,68
2,The 1941 Colima earthquake occurred on April 1...,Event,NaturalEvent,Earthquake,1941_Colima_earthquake,194
3,The 1983 Coalinga earthquake occurred on May 2...,Event,NaturalEvent,Earthquake,1983_Coalinga_earthquake,98
4,The 2013 Bushehr earthquake occurred with a mo...,Event,NaturalEvent,Earthquake,2013_Bushehr_earthquake,61


In [6]:
data.shape

(342781, 6)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342781 entries, 0 to 342780
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        342781 non-null  object
 1   l1          342781 non-null  object
 2   l2          342781 non-null  object
 3   l3          342781 non-null  object
 4   wiki_name   342781 non-null  object
 5   word_count  342781 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 15.7+ MB


In [9]:
data.isna().sum()

text          0
l1            0
l2            0
l3            0
wiki_name     0
word_count    0
dtype: int64

In [10]:
set(data['l1'])

{'Agent',
 'Device',
 'Event',
 'Place',
 'Species',
 'SportsSeason',
 'TopicalConcept',
 'UnitOfWork',
 'Work'}

In [11]:
data['text'][0]

'The 1994 Mindoro earthquake occurred on November 15 at 03:15 local time near Mindoro, the Philippines. It had a moment magnitude of 7.1. It is associated with a 35 kilometer-long ground rupture, called the Aglubang River fault. Seventy eight people were reported dead, and 7,566 houses were damaged. The earthquake generated a tsunami and landslides on the Verde Island.'

# Pythorch model

In [12]:
alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

In [13]:
args={'alphabet':alphabet,'number_of_characters':len(alphabet),'max_length':150,'dropout_input':0.3,'number_of_classes':len(set(data['l1']))}

In [14]:
args

{'alphabet': 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-,;.!?:\'"/\\|_@#$%^&*~`+-=<>()[]{}',
 'number_of_characters': 95,
 'max_length': 150,
 'dropout_input': 0.3,
 'number_of_classes': 9}

In [15]:
class CharacterLevelCNN(nn.Module):
    def __init__(self, args):
        super(CharacterLevelCNN, self).__init__()

        # define conv layers

        self.dropout_input = nn.Dropout2d(args['dropout_input'])

        self.conv1 = nn.Sequential(nn.Conv1d(args['number_of_characters'],
                                             256,
                                             kernel_size=7,
                                             padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(3)
                                   )

        self.conv2 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=7, padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(3)
                                   )

        self.conv3 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU()
                                   )

        self.conv4 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU()
                                   )

        self.conv5 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU()
                                   )

        self.conv6 = nn.Sequential(nn.Conv1d(256, 256, kernel_size=3, padding=0),
                                   nn.ReLU(),
                                   nn.MaxPool1d(3)
                                   )

        # compute the  output shape after forwarding an input to the conv layers

        input_shape = (128,
                       args['max_length'],
                       args['number_of_characters'])
        
        self.output_dimension = self._get_conv_output(input_shape)

        # define linear layers

        self.fc1 = nn.Sequential(
            nn.Linear(self.output_dimension, 1024),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.fc3 = nn.Linear(1024, args['number_of_classes'])

        # initialize weights

        self._create_weights()

    # utility private functions

    def _create_weights(self, mean=0.0, std=0.05):
        for module in self.modules():
            if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
                module.weight.data.normal_(mean, std)


    def _get_conv_output(self, shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        output_dimension = x.size(1)
        return output_dimension

    # forward

    def forward(self, x):
        x = self.dropout_input(x)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [16]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, args):
        self.texts = texts
        self.labels = labels
        self.length = len(self.texts)

        self.vocabulary = args['alphabet'] 
        self.number_of_characters = args['number_of_characters'] 
        self.max_length = args['max_length']
        self.identity_mat = np.identity(self.number_of_characters)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        raw_text = self.texts[index]

        data = np.array([self.identity_mat[self.vocabulary.index(i)] for i in list(raw_text)[::-1] if i in self.vocabulary],
                        dtype=np.float32)
        if len(data) > self.max_length:
            data = data[:self.max_length]
        elif 0 < len(data) < self.max_length:
            data = np.concatenate(
                (data, np.zeros((self.max_length - len(data), self.number_of_characters), dtype=np.float32)))
        elif len(data) == 0:
            data = np.zeros(
                (self.max_length, self.number_of_characters), dtype=np.float32)

        label = self.labels[index]
        data = torch.Tensor(data)

        return data, label

In [17]:
def remove_hashtags(text):
    clean_text = re.sub(r'#[A-Za-z0-9_]+', "", text)
    return clean_text

def remove_user_mentions(text):
    clean_text = re.sub(r'@[A-Za-z0-9_]+', "", text)
    return clean_text

def lower(text):
    return text.lower()

def remove_urls(text):
    clean_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return clean_text

preprocessing_steps = {
    'remove_hashtags': remove_hashtags,
    'remove_urls': remove_urls,
    'remove_user_mentions': remove_user_mentions,
    'lower': lower
}


def process_text(steps, text):
    if steps is not None:
        for step in steps:
            text = preprocessing_steps[step](text)
    return text

In [18]:
steps=['remove_hashtags','remove_urls','remove_user_mentions']

In [19]:
data['text']=data['text'].apply(lambda text: process_text(steps,text))

In [20]:
data['l1'].value_counts()/data.shape[0]*100

Agent             51.735948
Place             18.999886
Species            9.087143
Work               8.702933
Event              7.893961
SportsSeason       2.423413
UnitOfWork         0.728453
TopicalConcept     0.325281
Device             0.102981
Name: l1, dtype: float64

In [21]:
def labels_map(y):
    S=set(y)
    num=len(S)
    dic={word:i for i, word in enumerate(S)}
    
    labels=y.apply(lambda val: dic[val])
    
    return labels    

In [22]:
labels=labels_map(data['l1'])

In [23]:
type(labels)

pandas.core.series.Series

# Train

In [97]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    
    for epoch in range(1, epochs + 1):
        model.train() # Make sure that the model is in training mode.

        total_loss = 0

        for batch in train_loader:
            # get data
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()

            # get predictions from model
            y_pred = model(batch_x)
        
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()

        print("Epoch: {}, Loss: {}".format(epoch, total_loss / len(train_loader)))

In [24]:
from sklearn.model_selection import train_test_split

In [204]:
x_train,x_test,y_train,y_test=train_test_split(data['text'].values,labels.values,test_size=0.2)

In [206]:
dataset_train=MyDataset(x_train,y_train,args)

In [207]:
dataset_test=MyDataset(x_test,y_test,args)

In [209]:
test_dl=torch.utils.data.DataLoader(dataset_test, batch_size=128)
train_dl=torch.utils.data.DataLoader(dataset_train, batch_size=128)

## Focal Loss

In [ ]:
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int,long)): 
            self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): 
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, x, target):
        if x.dim()>2:
            x = x.view(x.size(0),x.size(1),-1)  # N,C,H,W => N,C,H*W
            x = x.transpose(1,2)    # N,C,H*W => N,H*W,C
            x = x.contiguous().view(-1,x.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(x)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=x.data.type():
                self.alpha = self.alpha.type_as(x.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: 
            return loss.mean()
        else: 
            return loss.sum()

## Initialize model and train

In [210]:
model=CharacterLevelCNN(number_classes,args)

In [211]:
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()

In [212]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device {}.".format(device))

Using device cpu.


In [ ]:
train(model, val_dl, 1, loss_fn,optimizer, device)

In [25]:
t = torch.tensor([[1,2],[3,4]])

In [26]:
t

tensor([[1, 2],
        [3, 4]])

In [31]:
torch.gather(t, 1, torch.tensor([[0,1],[0,1]]))

tensor([[1, 2],
        [3, 4]])